In [1]:
!pip install qiskit qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.6 MB/s eta 0:00:00


In [2]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_bloch_multivector
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def qft_rotations(qc, n):
    """Apply QFT rotations recursively on the first n qubits."""
    if n == 0:
        return qc
    n -= 1
    qc.h(n)
    for qubit in range(n):
        qc.cp(np.pi / 2**(n - qubit), qubit, n)
    qft_rotations(qc, n)
    return qc

In [4]:
def swap_registers(qc, n):
    """Swap qubits to reverse their order."""
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    return qc

In [5]:
def qft_circuit(n):
    """Builds an n-qubit Quantum Fourier Transform circuit."""
    qc = QuantumCircuit(n)
    qft_rotations(qc, n)
    swap_registers(qc, n)
    qc.name = "QFT"
    return qc

In [6]:
def run_qft(n):
    """Executes QFT circuit and displays Bloch sphere representation."""
    qc = qft_circuit(n)

    # Create a simulator and save the statevector explicitly
    simulator = AerSimulator(method='statevector')
    qc.save_statevector()

    # Transpile and run
    compiled = transpile(qc, simulator)
    result = simulator.run(compiled).result()

    # Retrieve statevector safely
    statevector = result.data(0)["statevector"]

    # Display results
    plot_bloch_multivector(statevector)
    plt.show()
    print(qc.draw(output='text'))

if __name__ == "__main__":
    n_qubits = 3
    print(f"Running Quantum Fourier Transform on {n_qubits} qubits...")
    run_qft(n_qubits)

Running Quantum Fourier Transform on 3 qubits...
                                          ┌───┐    statevector 
q_0: ──────■──────────────────────■───────┤ H ├─X───────░──────
           │                ┌───┐ │P(π/2) └───┘ │       ░      
q_1: ──────┼────────■───────┤ H ├─■─────────────┼───────░──────
     ┌───┐ │P(π/4)  │P(π/2) └───┘               │       ░      
q_2: ┤ H ├─■────────■───────────────────────────X───────░──────
     └───┘                                              ░      


In [8]:
#1. Vary the Number of Qubits
if __name__ == "__main__":
    for n_qubits in [2, 3, 4]:
        print(f"\nRunning QFT on {n_qubits} qubits...")
        run_qft(n_qubits)



Running QFT on 2 qubits...
                   ┌───┐    statevector 
q_0: ──────■───────┤ H ├─X───────░──────
     ┌───┐ │P(π/2) └───┘ │       ░      
q_1: ┤ H ├─■─────────────X───────░──────
     └───┘                       ░      

Running QFT on 3 qubits...
                                          ┌───┐    statevector 
q_0: ──────■──────────────────────■───────┤ H ├─X───────░──────
           │                ┌───┐ │P(π/2) └───┘ │       ░      
q_1: ──────┼────────■───────┤ H ├─■─────────────┼───────░──────
     ┌───┐ │P(π/4)  │P(π/2) └───┘               │       ░      
q_2: ┤ H ├─■────────■───────────────────────────X───────░──────
     └───┘                                              ░      

Running QFT on 4 qubits...
                                                                          ┌───┐»
q_0: ──────■───────────────────────────────■──────────────────────■───────┤ H ├»
           │                               │                ┌───┐ │P(π/2) └───┘»
q_1: ──────┼────────

In [10]:
#2. Inverse QFT
def inverse_qft_circuit(n):
    qc = QuantumCircuit(n)

    # Reverse of QFT: swap first to fix ordering
    swap_registers(qc, n)

    # Apply inverse rotations in reverse order
    for qubit in reversed(range(n)):
        for ctrl in reversed(range(qubit)):
            qc.cp(-np.pi / 2**(qubit - ctrl), ctrl, qubit)
        qc.h(qubit)

    qc.name = "QFT†"
    return qc
print(inverse_qft_circuit(3).draw(output='text'))


                                      ┌───┐
q_0: ─X────────────■─────────■────────┤ H ├
      │            │         │P(-π/2) ├───┤
q_1: ─┼──■─────────┼─────────■────────┤ H ├
      │  │P(-π/2)  │P(-π/4)   ┌───┐   └───┘
q_2: ─X──■─────────■──────────┤ H ├────────
                              └───┘        


In [22]:
#3. Integration with Phase Estimation
from qiskit import QuantumCircuit

def simple_phase_estimation(theta):
    qc = QuantumCircuit(3, 2)  # 2 counting qubits + 1 target qubit

    # Prepare eigenstate |1>
    qc.x(2)

    # Apply controlled-U operations
    qc.cp(theta, 1, 2)
    qc.cp(2*theta, 0, 2)

    # Apply QFT on counting qubits
    qc.append(qft_circuit(2), [0,1])

    qc.measure([0,1], [0,1])
    return qc
theta = np.pi / 4   # phase = 1/8 in fractional binary

qc = simple_phase_estimation(theta)

# Display circuit diagram
print(qc.draw(output='text'))


# Run the circuit on simulator
sim = AerSimulator()
compiled = transpile(qc, sim)
result = sim.run(compiled, shots=1024).result()

# Get and print measurement distribution
counts = result.get_counts()
print("Measurement Counts:", counts)


                            ┌──────┐┌─┐   
q_0: ───────────────■───────┤0     ├┤M├───
                    │       │  QFT │└╥┘┌─┐
q_1: ──────■────────┼───────┤1     ├─╫─┤M├
     ┌───┐ │P(π/4)  │P(π/2) └──────┘ ║ └╥┘
q_2: ┤ X ├─■────────■────────────────╫──╫─
     └───┘                           ║  ║ 
c: 2/════════════════════════════════╩══╩═
                                     0  1 
Measurement Counts: {'01': 271, '11': 255, '00': 250, '10': 248}


In [23]:
def run_qft_with_measurement(n):
    qc = qft_circuit(n)
    qc.measure_all()

    simulator = AerSimulator()
    compiled = transpile(qc, simulator)
    result = simulator.run(compiled, shots=1024).result()
    counts = result.get_counts()

    print(counts)
# Sample input for 3 qubits
run_qft_with_measurement(3)

{'000': 120, '010': 133, '101': 127, '100': 129, '110': 127, '001': 120, '111': 133, '011': 135}


In [ ]:
import matplotlib.pyplot as plt

# Force reinstallation of pylatexenc to ensure it's properly recognized
!pip install --force-reinstall pylatexenc


In [25]:
qc = qft_circuit(3)
print(qc.draw(output='text'))

                                          ┌───┐   
q_0: ──────■──────────────────────■───────┤ H ├─X─
           │                ┌───┐ │P(π/2) └───┘ │ 
q_1: ──────┼────────■───────┤ H ├─■─────────────┼─
     ┌───┐ │P(π/4)  │P(π/2) └───┘               │ 
q_2: ┤ H ├─■────────■───────────────────────────X─
     └───┘                                        
